In [27]:
import torch

# see if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available")

# see if cuDNN is available
if torch.backends.cudnn.enabled:
    print("cuDNN is available")

# set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is set to:", device)

CUDA is available
cuDNN is available
Device is set to: cuda


In [28]:
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning import Trainer, loggers
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import torchmetrics
import tensorboard

class CIFAR100DataModule(pl.LightningDataModule):
    def __init__(self, data_dir='./data', batch_size=128):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
        ])

    def setup(self, stage=None):
        # Load data
        cifar100_train = datasets.CIFAR100(root=self.data_dir, train=True, download=True, transform=self.transform)
        cifar100_test = datasets.CIFAR100(root=self.data_dir, train=False, download=True, transform=self.transform)

        # Train/val split
        self.train, self.val = random_split(cifar100_train, [45000, 5000])
        self.test = cifar100_test

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, shuffle=True, num_workers=7, persistent_workers=True)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size, num_workers=7, persistent_workers=True)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size, num_workers=7, persistent_workers=True)


In [33]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.relu = nn.ReLU(inplace=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * 4:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * 4, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * 4)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out
        

class ResNet(pl.LightningModule):
    def __init__(self, block, num_blocks, num_classes=100, learning_rate=0.01):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.learning_rate = learning_rate

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # Accuracy metrics
        self.train_acc = torchmetrics.Accuracy(num_classes=num_classes, average='macro', task='multiclass')
        self.val_acc = torchmetrics.Accuracy(num_classes=num_classes, average='macro', task='multiclass')

        # Testing metrics
        self.test_acc = torchmetrics.Accuracy(num_classes=num_classes, average='macro', task='multiclass')
        self.test_precision = torchmetrics.Precision(num_classes=num_classes, average='macro', task='multiclass')
        self.test_recall = torchmetrics.Recall(num_classes=num_classes, average='macro', task='multiclass')
        self.test_f1 = torchmetrics.F1Score(num_classes=num_classes, average='macro', task='multiclass')

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.train_acc(logits, y)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        self.log('train_acc', self.train_acc, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.val_acc(logits, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', self.val_acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        # Update metrics
        self.test_acc(logits, y)
        self.test_precision(logits, y)
        self.test_recall(logits, y)
        self.test_f1(logits, y)

        self.log('test_loss', loss)
        self.log('test_acc', self.test_acc, prog_bar=True)
        self.log('test_precision', self.test_precision, prog_bar=True)
        self.log('test_recall', self.test_recall, prog_bar=True)
        self.log('test_f1', self.test_f1, prog_bar=True)
        return {"loss": loss}

def ResNet200(num_classes=100, learning_rate=0.01):
    return ResNet(Bottleneck, [3, 24, 36, 3], num_classes, learning_rate)

In [30]:
early_stopping = EarlyStopping(
    monitor='val_acc',
    min_delta=0.00,
    patience=10,
    verbose=True,
    mode='max'
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_acc',
    filename='best-checkpoint',
    save_top_k=1,
    mode='max', 
    verbose=True
)

data_module = CIFAR100DataModule()
model = ResNet200(num_classes=100, learning_rate=0.001)
logger = TensorBoardLogger('C:/Users/Owner/Documents/MMA/MULTI TASK/Final Project/', name='cnn_logs')
trainer = pl.Trainer(max_epochs=500, devices=1 if torch.cuda.is_available() else 0, callbacks=[early_stopping, checkpoint_callback])
trainer.fit(model, datamodule=data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name           | Type                | Params
--------------------------------------------------------
0  | conv1          | Conv2d              | 1.7 K 
1  | bn1            | BatchNorm2d         | 128   
2  | relu           | ReLU                | 0     
3  | layer1         | Sequential          | 215 K 
4  | layer2         | Sequential          | 6.8 M 
5  | layer3         | Sequential          | 40.6 M
6  | layer4         | Sequential          | 15.0 M
7  | avgpool        | AdaptiveAvgPool2d   | 0     
8  | fc             | Linear              | 204 K 
9  | train_acc      | MulticlassAccuracy  | 0     
10 | val_acc        | MulticlassAccuracy  | 0     
11 | test_acc       | MulticlassAccuracy  | 0     
12 | test_precision | MulticlassPrecision | 0     
13 | test_recall    | MulticlassRecall    | 0     
14 | test_f1        | MulticlassF1Score   | 0     
--------------------------------------------------------
62.8 M    Trainable params


Epoch 0: 100%|██████████| 352/352 [1:00:22<00:00,  0.10it/s, v_num=62, train_loss_step=3.900, train_acc_step=0.0873, val_loss=4.020, val_acc=0.0735, train_loss_epoch=4.340, train_acc_epoch=0.041]

Metric val_acc improved. New best score: 0.073
Epoch 0, global step 352: 'val_acc' reached 0.07350 (best 0.07350), saving model to 'c:\\Users\\Owner\\Documents\\MMA\\MULTI TASK\\Final Project\\lightning_logs\\version_62\\checkpoints\\best-checkpoint.ckpt' as top 1


...

Epoch 46, global step 16544: 'val_acc' was not in top 1


Epoch 47: 100%|██████████| 352/352 [51:11<00:00,  0.11it/s, v_num=62, train_loss_step=0.147, train_acc_step=0.918, val_loss=2.920, val_acc=0.533, train_loss_epoch=0.128, train_acc_epoch=0.960]  

Monitored metric val_acc did not improve in the last 10 records. Best score: 0.556. Signaling Trainer to stop.
Epoch 47, global step 16896: 'val_acc' was not in top 1


Epoch 47: 100%|██████████| 352/352 [51:11<00:00,  0.11it/s, v_num=62, train_loss_step=0.147, train_acc_step=0.918, val_loss=2.920, val_acc=0.533, train_loss_epoch=0.128, train_acc_epoch=0.960]


AttributeError: 'function' object has no attribute 'load_from_checkpoint'

In [34]:
def load_resnet200_from_checkpoint(checkpoint_path, num_classes=100, learning_rate=0.01):
    model = ResNet200(num_classes=num_classes, learning_rate=learning_rate)
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    return model


best_model_path = checkpoint_callback.best_model_path
best_model = load_resnet200_from_checkpoint(best_model_path, num_classes=100, learning_rate=0.01)
trainer.test(best_model, datamodule=data_module)

Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 79/79 [00:09<00:00,  8.53it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5537999868392944     │
│          test_f1          │    0.5511781573295593     │
│         test_loss         │     2.614758253097534     │
│      test_precision       │    0.5764963626861572     │
│        test_recall        │    0.5537999868392944     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2.614758253097534,
  'test_acc': 0.5537999868392944,
  'test_precision': 0.5764963626861572,
  'test_recall': 0.5537999868392944,
  'test_f1': 0.5511781573295593}]

In [40]:
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, data_dir='./data', batch_size=128):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])

    def setup(self, stage=None):
        # Load data
        cifar10_train = datasets.CIFAR10(root=self.data_dir, train=True, download=True, transform=self.transform)
        cifar10_test = datasets.CIFAR10(root=self.data_dir, train=False, download=True, transform=self.transform)

        # Train/val split
        self.train, self.val = random_split(cifar10_train, [45000, 5000])
        self.test = cifar10_test

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, shuffle=True, num_workers=7, persistent_workers=True)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size, num_workers=7, persistent_workers=True)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size, num_workers=7, persistent_workers=True)



def ResNet50(num_classes=10, learning_rate=0.01):
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes, learning_rate)

In [41]:
early_stopping = EarlyStopping(
    monitor='val_acc',
    min_delta=0.00,
    patience=10,
    verbose=True,
    mode='max'
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_acc',
    filename='best-checkpoint',
    save_top_k=1,
    mode='max', 
    verbose=True
)

data_module = CIFAR10DataModule()
model = ResNet50(num_classes=10, learning_rate=0.001)
logger = TensorBoardLogger('C:/Users/Owner/Documents/MMA/MULTI TASK/Final Project/', name='cnn2_logs')
trainer = pl.Trainer(max_epochs=500, devices=1 if torch.cuda.is_available() else 0, callbacks=[early_stopping, checkpoint_callback])
trainer.fit(model, datamodule=data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name           | Type                | Params
--------------------------------------------------------
0  | conv1          | Conv2d              | 1.7 K 
1  | bn1            | BatchNorm2d         | 128   
2  | relu           | ReLU                | 0     
3  | layer1         | Sequential          | 215 K 
4  | layer2         | Sequential          | 1.2 M 
5  | layer3         | Sequential          | 7.1 M 
6  | layer4         | Sequential          | 15.0 M
7  | avgpool        | AdaptiveAvgPool2d   | 0     
8  | fc             | Linear              | 20.5 K
9  | train_acc      | MulticlassAccuracy  | 0     
10 | val_acc        | MulticlassAccuracy  | 0     
11 | test_acc       | MulticlassAccuracy  | 0     
12 | test_precision | MulticlassPrecision | 0     
13 | test_recall    | MulticlassRecall    | 0     
14 | test_f1        | MulticlassF1Score   | 0     
--------------------------------------------------------
23.5 M    Trainable params


Epoch 0: 100%|██████████| 352/352 [00:42<00:00,  8.35it/s, v_num=64, train_loss_step=0.906, train_acc_step=0.711, val_loss=1.200, val_acc=0.563, train_loss_epoch=1.540, train_acc_epoch=0.442]

Metric val_acc improved. New best score: 0.563
Epoch 0, global step 352: 'val_acc' reached 0.56255 (best 0.56255), saving model to 'c:\\Users\\Owner\\Documents\\MMA\\MULTI TASK\\Final Project\\lightning_logs\\version_64\\checkpoints\\best-checkpoint.ckpt' as top 1


...

Epoch 52: 100%|██████████| 352/352 [00:40<00:00,  8.75it/s, v_num=64, train_loss_step=0.0391, train_acc_step=0.995, val_loss=1.010, val_acc=0.831, train_loss_epoch=0.0224, train_acc_epoch=0.992]  

Epoch 52, global step 18656: 'val_acc' was not in top 1


Epoch 53: 100%|██████████| 352/352 [00:40<00:00,  8.74it/s, v_num=64, train_loss_step=0.00162, train_acc_step=1.000, val_loss=0.899, val_acc=0.847, train_loss_epoch=0.0243, train_acc_epoch=0.992] 

Monitored metric val_acc did not improve in the last 10 records. Best score: 0.866. Signaling Trainer to stop.
Epoch 53, global step 19008: 'val_acc' was not in top 1


Epoch 53: 100%|██████████| 352/352 [00:40<00:00,  8.74it/s, v_num=64, train_loss_step=0.00162, train_acc_step=1.000, val_loss=0.899, val_acc=0.847, train_loss_epoch=0.0243, train_acc_epoch=0.992]


In [44]:
def load_resnet50_from_checkpoint(checkpoint_path, num_classes=10, learning_rate=0.01):
    model = ResNet50(num_classes=num_classes, learning_rate=learning_rate)
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    return model


best_model_path = checkpoint_callback.best_model_path
best_model = load_resnet50_from_checkpoint(best_model_path, num_classes=10, learning_rate=0.01)
trainer.test(best_model, datamodule=data_module)

Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 79/79 [00:03<00:00, 23.26it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8579999804496765     │
│          test_f1          │    0.8572114109992981     │
│         test_loss         │    0.8349453806877136     │
│      test_precision       │    0.8574167490005493     │
│        test_recall        │    0.8579999804496765     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.8349453806877136,
  'test_acc': 0.8579999804496765,
  'test_precision': 0.8574167490005493,
  'test_recall': 0.8579999804496765,
  'test_f1': 0.8572114109992981}]